In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import torch
import itertools
import string
import numpy as np
from IPython.display import clear_output
from matplotlib import pyplot as plt
from gensim.models import word2vec
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

from sklearn import metrics
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_difficult_26_01.csv')
target_names = list(set(df['doc_name'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)
df['doc_name_enc']=df['doc_name'].replace(label2idx)
df

{'Указ, федеральный уровень, глава исполнительной власти': 0, 'Постановление, региональный уровень, глава исполнительной власти': 1, 'Закон, региональный уровень, законодательный орган': 2, 'Постановление, региональный уровень, законодательный орган': 3, 'Распоряжение, федеральный уровень, глава исполнительной власти': 4, 'Распоряжение, региональный уровень, глава исполнительной власти': 5, 'Постановление, региональный уровень, органы исполнительной власти': 6, 'Постановление, региональный уровень, правительство': 7, 'Закон, федеральный уровень, законодательный орган': 8, 'Распоряжение, федеральный уровень, правительство': 9, 'Приказ, федеральный уровень, органы исполнительной власти': 10, 'Указ, региональный уровень, глава исполнительной власти': 11, 'Распоряжение, региональный уровень, правительство': 12, 'Приказ, региональный уровень, органы исполнительной власти': 13, 'Постановление, федеральный уровень, правительство': 14}


,doc_name,doc_text,text_token,text_token_filtred,text_difficult,doc_name_enc
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация ус...,распоряжение президент российский федерация ус...,комитет российский федерация установить числен...,4
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация пр...,распоряжение президент российский федерация пр...,российский федерация контроль оборот наркотик ...,4
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...,указ президент российский федерация освобожден...,указ президент российский федерация освобожден...,федерация контроль оборот наркотик освободить ...,0
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,многолетний плодотворный деятельность наградит...,4
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,молодёжь объявить благодарность андропов игорь...,4
...,...,...,...,...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...,губернатор орловский область мера проведение к...,губернатор орловский область мера проведение к...,проживать инвалид ветеран великий отечественны...,11
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ а г х м башкортостан республи...,ммжхжчмитжкъщий башкортостан республикаь респу...,ммжхжчмитжкъщий башкортостан республикаь респу...,должность заместитель премьер министр правител...,11
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края об отмене...,губернатор пермский край отмена ограничение ка...,губернатор пермский край отмена ограничение ка...,посёлок октябрьский соответствие закон российс...,11
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края г об утве...,губернатор пермский край утверждение порядок п...,губернатор пермский край утверждение порядок п...,должность пермский край отдельный должность го...,11


In [ ]:
import gensim.downloader as api
print("\n".join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [ ]:
word2vec_pre = api.load("glove-twitter-50")
tok = df['text_token_filtred'][:3].str.split().tolist()
word2vec_pre.most_similar(positive=['президент'])

[==================================================] 100.0% 199.5/199.5MB downloaded


[('премьер-министр', 0.864189863204956),
 ('путин', 0.8365647196769714),
 ('глава', 0.8298710584640503),
 ('заявил', 0.8257015943527222),
 ('мид', 0.8176457285881042),
 ('владимир', 0.8131812810897827),
 ('посол', 0.810960590839386),
 ('украины', 0.8079652190208435),
 ('янукович', 0.8058887720108032),
 ('грузии', 0.8036628365516663)]

Своя модель

In [ ]:
texts = df['text_difficult'].tolist()
words = [i.split() for i in texts]
count_words = Counter()
for text in words:
  for i in text:
    count_words[i]+=1
more_10 = [k for k, v in count_words.items() if v>10]
words_more_10 = []
for t in words:
  passed = [i for i in t if i in more_10]
  words_more_10.append(passed)
X = np.array(words_more_10)
model_wv_dif = word2vec.Word2Vec(X, vector_size=50, window=5, min_count=1, workers=4)
model_wv_dif.wv.most_similar(positive=['президент'])

<ipython-input-5-31f3a9fa9ad4>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(words_more_10)


[('глава', 0.5370832681655884),
 ('правительство', 0.5321834087371826),
 ('губернатор', 0.4876767694950104),
 ('федерация', 0.44561415910720825),
 ('коррупция', 0.4324455261230469),
 ('противодействие', 0.4177606999874115),
 ('палата', 0.4115430414676666),
 ('российский', 0.4110330045223236),
 ('ndот', 0.4023815989494324),
 ('август', 0.3951947093009949)]

In [ ]:

tfidf_dif  = TfidfVectorizer()
tfidf_dif.fit([' '.join(i) for i in df['text_difficult'].str.split()])
tf_dict_dif=dict(zip(tfidf_dif.get_feature_names_out(), tfidf_dif.idf_))

In [ ]:
def average_emb(w2v, text):

    doc = [word for word in text.split() if word in w2v.wv]

    return np.mean(w2v.wv[doc], axis=0)

def average_emb_tdidf(w2v, td_dict, text):

    doc = [word for word in text.split() if word in w2v.wv]
    tf = np.array([td_dict[word] for word in text.split() if word in w2v.wv])
    #print(w2v.wv[doc].dot(tf[0]))
    return np.mean(w2v.wv[doc].dot(tf[0]), axis=0)

def average_emb_pre(w2v, text):

    doc = [word for word in text.split() if word in w2v]

    return np.mean(w2v[doc], axis=0)

def average_emb_tdidf_pre(w2v, td_dict, text):

    doc = [word for word in text.split() if word in w2v]
    tf = np.array([td_dict[word] for word in text.split() if word in w2v])

    return np.mean(w2v[doc].dot(tf[0]), axis=0)

In [ ]:
df_w2v = df[['doc_name', 'text_difficult']].copy()

df_w2v['w2v_dif'] = df_w2v.apply(lambda x: average_emb(model_wv_dif, x['text_difficult']),axis=1)
df_w2v['w2v_dif_idf'] = df_w2v.apply(lambda x: average_emb_tdidf(model_wv_dif,tf_dict_dif, x['text_difficult']),axis=1)

df_w2v['w2v_dif_pre'] = df_w2v.apply(lambda x: average_emb_pre(word2vec_pre, x['text_difficult']),axis=1)
df_w2v['w2v_dif_idf_pre'] = df_w2v.apply(lambda x: average_emb_tdidf_pre(word2vec_pre,tf_dict_dif, x['text_difficult']),axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.6545 {'lr_clf__class_weight': None, 'lr_clf__penalty': 'l2', 'lr_clf__solver': 'lbfgs'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='lbfgs',penalty='l2', class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('lr_clf', LogisticRegression(random_state=42))])

In [ ]:
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.83      0.84      0.83       329
                Закон, федеральный уровень, законодательный орган       0.91      0.92      0.91       347
 Постановление, региональный уровень, глава исполнительной власти       0.56      0.50      0.53       318
       Постановление, региональный уровень, законодательный орган       0.89      0.89      0.89       302
Постановление, региональный уровень, органы исполнительной власти       0.88      0.87      0.87       244
               Постановление, региональный уровень, правительство       0.62      0.61      0.62       318
                Постановление, федеральный уровень, правительство       0.74      0.80      0.77       314
       Приказ, региональный уровень, органы исполнительной власти       0.71      0.72      0.72       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Best score and parameter combination:
0.571 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'log', 'sgd_clf__penalty': 'elasticnet'}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='log_loss',penalty='elasticnet', class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.78      0.79      0.78       329
                Закон, федеральный уровень, законодательный орган       0.89      0.91      0.90       347
 Постановление, региональный уровень, глава исполнительной власти       0.55      0.54      0.55       318
       Постановление, региональный уровень, законодательный орган       0.83      0.90      0.86       302
Постановление, региональный уровень, органы исполнительной власти       0.87      0.88      0.88       244
               Постановление, региональный уровень, правительство       0.51      0.63      0.56       318
                Постановление, федеральный уровень, правительство       0.70      0.82      0.75       314
       Приказ, региональный уровень, органы исполнительной власти       0.86      0.60      0.71       325
        Приказ, федеральный уровень,

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.638 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.74      0.83      0.78       329
                Закон, федеральный уровень, законодательный орган       0.88      0.90      0.89       347
 Постановление, региональный уровень, глава исполнительной власти       0.59      0.60      0.60       318
       Постановление, региональный уровень, законодательный орган       0.94      0.82      0.87       302
Постановление, региональный уровень, органы исполнительной власти       0.95      0.86      0.90       244
               Постановление, региональный уровень, правительство       0.46      0.64      0.54       318
                Постановление, федеральный уровень, правительство       0.72      0.82      0.76       314
       Приказ, региональный уровень, органы исполнительной власти       0.80      0.66      0.73       325
        Приказ, федеральный уровень,

Вместе с td-idf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_idf'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.64 {'lr_clf__class_weight': 'balanced', 'lr_clf__penalty': 'l1', 'lr_clf__solver': 'saga'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='saga',penalty='l1', class_weight='balanced', random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.81      0.83      0.82       329
                Закон, федеральный уровень, законодательный орган       0.90      0.91      0.91       347
 Постановление, региональный уровень, глава исполнительной власти       0.57      0.54      0.56       318
       Постановление, региональный уровень, законодательный орган       0.87      0.88      0.87       302
Постановление, региональный уровень, органы исполнительной власти       0.85      0.88      0.86       244
               Постановление, региональный уровень, правительство       0.60      0.61      0.61       318
                Постановление, федеральный уровень, правительство       0.70      0.81      0.75       314
       Приказ, региональный уровень, органы исполнительной власти       0.71      0.72      0.71       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.538 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': None}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='hinge',penalty=None, class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.63      0.80      0.70       329
                Закон, федеральный уровень, законодательный орган       0.86      0.85      0.86       347
 Постановление, региональный уровень, глава исполнительной власти       0.47      0.23      0.31       318
       Постановление, региональный уровень, законодательный орган       0.82      0.82      0.82       302
Постановление, региональный уровень, органы исполнительной власти       0.82      0.84      0.83       244
               Постановление, региональный уровень, правительство       0.61      0.04      0.08       318
                Постановление, федеральный уровень, правительство       0.59      0.71      0.65       314
       Приказ, региональный уровень, органы исполнительной власти       0.43      0.72      0.54       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6164999999999999 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.71      0.81      0.76       329
                Закон, федеральный уровень, законодательный орган       0.85      0.87      0.86       347
 Постановление, региональный уровень, глава исполнительной власти       0.58      0.60      0.59       318
       Постановление, региональный уровень, законодательный орган       0.90      0.83      0.86       302
Постановление, региональный уровень, органы исполнительной власти       0.92      0.86      0.89       244
               Постановление, региональный уровень, правительство       0.43      0.62      0.50       318
                Постановление, федеральный уровень, правительство       0.64      0.78      0.70       314
       Приказ, региональный уровень, органы исполнительной власти       0.84      0.68      0.75       325
        Приказ, федеральный уровень,

Преобученный

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_pre'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.6034999999999999 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'saga'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='saga',penalty=None, class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.73      0.80      0.77       329
                Закон, федеральный уровень, законодательный орган       0.85      0.84      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.52      0.47      0.50       318
       Постановление, региональный уровень, законодательный орган       0.76      0.76      0.76       302
Постановление, региональный уровень, органы исполнительной власти       0.85      0.85      0.85       244
               Постановление, региональный уровень, правительство       0.52      0.52      0.52       318
                Постановление, федеральный уровень, правительство       0.66      0.72      0.68       314
       Приказ, региональный уровень, органы исполнительной власти       0.65      0.70      0.67       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Best score and parameter combination:
0.5485 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'log', 'sgd_clf__penalty': 'l1'}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='log_loss',penalty='l1', class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.68      0.80      0.74       329
                Закон, федеральный уровень, законодательный орган       0.82      0.86      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.49      0.14      0.21       318
       Постановление, региональный уровень, законодательный орган       0.73      0.76      0.75       302
Постановление, региональный уровень, органы исполнительной власти       0.88      0.84      0.86       244
               Постановление, региональный уровень, правительство       0.60      0.31      0.40       318
                Постановление, федеральный уровень, правительство       0.68      0.68      0.68       314
       Приказ, региональный уровень, органы исполнительной власти       0.59      0.68      0.63       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.65      0.84      0.73       329
                Закон, федеральный уровень, законодательный орган       0.82      0.86      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.65      0.51      0.57       318
       Постановление, региональный уровень, законодательный орган       0.85      0.76      0.80       302
Постановление, региональный уровень, органы исполнительной власти       0.94      0.86      0.90       244
               Постановление, региональный уровень, правительство       0.48      0.58      0.52       318
                Постановление, федеральный уровень, правительство       0.65      0.80      0.71       314
       Приказ, региональный уровень, органы исполнительной власти       0.69      0.72      0.70       325
        Приказ, федеральный уровень,

Предобученный с td-idf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_idf_pre'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.58 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'sag'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='sag',penalty=None, class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.70      0.79      0.74       329
                Закон, федеральный уровень, законодательный орган       0.83      0.84      0.83       347
 Постановление, региональный уровень, глава исполнительной власти       0.53      0.44      0.48       318
       Постановление, региональный уровень, законодательный орган       0.73      0.75      0.74       302
Постановление, региональный уровень, органы исполнительной власти       0.83      0.86      0.85       244
               Постановление, региональный уровень, правительство       0.50      0.56      0.53       318
                Постановление, федеральный уровень, правительство       0.62      0.72      0.67       314
       Приказ, региональный уровень, органы исполнительной власти       0.65      0.68      0.66       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.52 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'squared_hinge', 'sgd_clf__penalty': None}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='squared_hinge',penalty=None, class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.32      0.71      0.44       329
                Закон, федеральный уровень, законодательный орган       0.76      0.79      0.77       347
 Постановление, региональный уровень, глава исполнительной власти       0.31      0.58      0.40       318
       Постановление, региональный уровень, законодательный орган       0.66      0.40      0.50       302
Постановление, региональный уровень, органы исполнительной власти       0.76      0.75      0.76       244
               Постановление, региональный уровень, правительство       0.51      0.29      0.37       318
                Постановление, федеральный уровень, правительство       0.56      0.64      0.60       314
       Приказ, региональный уровень, органы исполнительной власти       0.25      0.63      0.36       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.5415000000000001 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.59      0.76      0.66       329
                Закон, федеральный уровень, законодательный орган       0.76      0.87      0.81       347
 Постановление, региональный уровень, глава исполнительной власти       0.59      0.49      0.54       318
       Постановление, региональный уровень, законодательный орган       0.79      0.74      0.76       302
Постановление, региональный уровень, органы исполнительной власти       0.94      0.84      0.88       244
               Постановление, региональный уровень, правительство       0.41      0.50      0.45       318
                Постановление, федеральный уровень, правительство       0.65      0.74      0.69       314
       Приказ, региональный уровень, органы исполнительной власти       0.63      0.70      0.66       325
        Приказ, федеральный уровень,